### 작업형 제1유형

#### 1.

In [14]:
import pandas as pd

df = pd.read_csv('../yemoonsaBigdata/datasets/Part3/201_boston.csv')

sorted_df = df.sort_values(by='CRIM', ascending=False)
sorted_df = sorted_df.reset_index(drop=True)
top_10_crim = sorted_df.loc[9, 'CRIM']

sorted_df.loc[0:9, 'CRIM'] = top_10_crim
print(sorted_df[sorted_df['AGE'] >= 80]['CRIM'].mean())

5.759386625


#### 2.

In [23]:
import pandas as pd

df = pd.read_csv('../yemoonsaBigdata/datasets/Part3/202_housing.csv')
per = int(len(df) * 0.8)
new_df = df.iloc[:per, :]

prev_std = new_df['total_bedrooms'].std()

med = new_df['total_bedrooms'].median()
new_df.loc[:, 'total_bedrooms'] = new_df['total_bedrooms'].fillna(med)
curr_std = new_df['total_bedrooms'].std()

print(abs(prev_std - curr_std))



1.9751472916456692


#### 3.

In [29]:
import pandas as pd

df = pd.read_csv('../yemoonsaBigdata/datasets/Part3/202_housing.csv')

mean = df['population'].mean()
std = df['population'].std()

upper_outlier = mean + 1.5 * std
lower_outlier = mean - 1.5 * std

print(df[(df['population'] > upper_outlier) | (df['population'] < lower_outlier)]['population'].sum())

5607295.0


### 작업형 제2유형

In [47]:
import pandas as pd
import numpy as np

X_train = pd.read_csv('../yemoonsaBigdata/datasets/Part3/204_x_train.csv')
y_train = pd.read_csv('../yemoonsaBigdata/datasets/Part3/204_y_train.csv')
X_test = pd.read_csv('../yemoonsaBigdata/datasets/Part3/204_x_test.csv')


y_cols = ['Reached.on.Time_Y.N']
num_cols = ['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']
cat_cols = ['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']


# 범주형 데이터 전처리
from sklearn.preprocessing import LabelEncoder
X = pd.concat([X_train, X_test])

for col in cat_cols:
    label_encoder = LabelEncoder()
    label_encoder.fit(X[col])
    
    X_train[col] = label_encoder.transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])
    
    
# 학습, 검증 데이터셋 분리
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train[num_cols+cat_cols], y_train[y_cols], test_size=0.3, stratify=y_train[y_cols])


# 수치형 데이터 전처리
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[num_cols])

X_tr[num_cols] = scaler.transform(X_tr[num_cols])
X_val[num_cols] = scaler.transform(X_val[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])


# 모델 학습
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

'''
model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.squeeze())
y_pred = model_rf.predict_proba(X_val)
print(roc_auc_score(y_val, y_pred[:, 1]))

model_xgb = XGBClassifier()
model_xgb.fit(X_tr, y_tr.values.squeeze())
y_pred = model_xgb.predict_proba(X_val)
print(roc_auc_score(y_val, y_pred[:, 1]))
'''

model_xgb1 = XGBClassifier(n_estimators=50, max_depth=5)
model_xgb1.fit(X_tr, y_tr.values.squeeze())
# y_pred = model_xgb1.predict_proba(X_val)
# print(roc_auc_score(y_val, y_pred[:, 1]))


y_pred = model_xgb1.predict_proba(X_test[num_cols+cat_cols])
y_pred = y_pred[:, 1]


result = pd.DataFrame({
    'index': X_test['ID'],
    'y_pred': y_pred
})

print(result)
result.to_csv('../yemoonsaBigdata/res/4.csv', index=False)


      index    y_pred
0      7007  0.492721
1      9793  0.554909
2      6593  0.507808
3     10527  0.288298
4      6914  0.438892
...     ...       ...
4395   3142  0.264118
4396   6948  0.357000
4397   7531  0.455983
4398   8398  0.365105
4399  10252  0.608149

[4400 rows x 2 columns]
